In [1]:
import pandas as pd
import numpy as np
import gc
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
# x = pd.read_csv("x.csv")
# j = pd.read_csv("j.csv")
# x = x[x['WIFIAPTag'].apply(lambda x:x[:2] in ['W1', 'W2', 'W3', 'E1', 'E2', 'E3', 'T1', 'TC', 'EC'])]
# j = j[j['WIFIAPTag'].apply(lambda x:x[:2] in ['W1', 'W2', 'W3', 'E1', 'E2', 'E3', 'T1', 'TC', 'EC'])]
# mer = pd.merge(x, j, on=['WIFIAPTag', 'slice10min'])
# mer['passengerCount'] = (mer['passengerCount_x'] + mer['passengerCount_y'])/float(2)
# mer.to_csv('sub.csv', columns=['WIFIAPTag', 'slice10min', 'passengerCount'])

In [3]:
offline = 1
start = 0
slic = 10

In [4]:
#  wifi = pd.read_csv("data/WIFI_AP_Passenger_Records_chusai_2ndround.csv")
# wifi['timeStamp'] = wifi['timeStamp'].apply(lambda x:'/'.join(x.split('-')[:3]) + ' ' + ':'.join(x.split('-')[3:]))
# wifi.to_csv("data/wifi_ap2.csv", index=False)

In [5]:
# depart = pd.read_csv("data/airport_gz_departure_chusai_2stround.csv", parse_dates=[2,3])
wifi = pd.read_csv("data/wifi_ap2.csv", parse_dates=[2])
wifi = wifi[wifi['timeStamp']>='2016-09-12 00:00:00']
#缺失的８个ｔａｇ可以用规则
wifi = wifi[wifi['WIFIAPTag'].apply(lambda x:x not in ['E2-3B<E2-3-09>','E2-3B<E2-3-10>', 'E2-3B<E2-3-11>', 'E2-3B<E2-3-12>',
 'E2-3B<E2-3-13>', 'E2-3B<E2-3-14>', 'E2-3B<E2-3-15>', 'E2-3B<E2-3-16>'])]
# if offline==1:
#     wifi = wifi[wifi['timeStamp']<'2016-09-24 15:00:00']
# if offline==2:
#     wifi = wifi[wifi['timeStamp']<'2016-09-25 12:00:00']
# wifi['area'] = wifi['WIFIAPTag'].apply(lambda x:x[:2])
# wifi = wifi[wifi['area'].apply(lambda x:x[:2] in ['W1', 'W2', 'W3', 'E1', 'E2', 'E3', 'T1', 'TC', 'EC'])]
# wifi['mark'] = wifi['WIFIAPTag'].apply(lambda x:x[4:5])
# wifi['floor'] = wifi['WIFIAPTag'].apply(lambda x:x[3:4])
# wifi['num_infloor'] = wifi['WIFIAPTag'].apply(lambda x:x[-3:-1])
# wifi['num_incurr'] = wifi['WIFIAPTag'].apply(lambda x:x[6:7])

In [7]:
#749个wifitag,588个需要预测的
#749*18=
#wifi Timestamp('2016-09-10 18:55:04')-Timestamp('2016-09-14 14:59:01')
#4*6*6 = 144
from datetime import timedelta
date = [pd.date_range('2016-09-12 12:00:00', '2016-09-12 18:00:00', freq=str(slic)+'T')[:-1]]
for i in range(13):
    date.append(date[-1] + timedelta(1))
    
#从２１号开始    
if start==21:
    date = [pd.date_range('2016-09-21 12:00:00', '2016-09-21 18:00:00', freq=str(slic)+'T')[:-1]]
    for i in range(4):
        date.append(date[-1] + timedelta(1))
    

if offline==1:
    del date[-1]
if offline==2:
    date = [d-timedelta(0,3*60*60) for d in date]

date_lst = []
for i in range(len(date)):
    date_lst.extend(date[i])
    date_lst.extend(date[i][180/slic:] + timedelta(0,3*60*60))

In [8]:
grouped = wifi.groupby('WIFIAPTag')
g_lst = []
for tag, group in grouped:
    group = group.set_index('timeStamp').resample(str(slic)+'T').mean().reindex(date_lst).fillna(0)
    group['tag'] = tag
    g_lst.append(group)

wifi_mean = pd.concat(g_lst).reset_index()

In [9]:
wifi_2D = wifi_mean.set_index(['timeStamp', 'tag']).unstack()['passengerCount']
wifi_corr = wifi_2D.corr()

In [10]:
wifi_mean['slice'] = wifi_mean['timeStamp'].apply(lambda x:x.hour*6+x.minute/10)

wifi_slice_mean = wifi_mean[wifi_mean['timeStamp']<date[-1][1]].groupby(['tag', 'slice']).mean().reset_index()
wifi_slice_mean.columns = ['tag', 'slice', 'slice_mean']

wifi_slice_std = wifi_mean[wifi_mean['timeStamp']<date[-1][1]].groupby(['tag', 'slice']).std().reset_index()
wifi_slice_std.columns = ['tag', 'slice', 'slice_std']

wifi_mean = pd.merge(wifi_mean, wifi_slice_mean, how='left').fillna(0)
wifi_mean = pd.merge(wifi_mean, wifi_slice_std, how='left').fillna(0)

In [11]:
data = wifi_mean[['timeStamp','passengerCount','tag']].copy()
data_copy = data.copy()
data_copy.columns = ['time', 'count', 'tag']

In [12]:
#前面100/slic个时间片
for i in range(-100/slic, 0):
#180/slic+1
    s = 'pre_'+str(i*slic)+'_min'
    data[s] = data['timeStamp']+timedelta(0, i*slic*60)
#     print data[['timeStamp', s]]
    new_data = pd.merge(data, data_copy, left_on=['tag',s], right_on=['tag','time'], how='left')
    new_data[s] = new_data['count']
    data = new_data.drop(['time', 'count'], axis=1)
    del new_data
    gc.collect()

#前一天
for i in range(-8,9):
    s = 'pre_1_day'+str(i*slic)+'_min'
    data[s] = data['timeStamp'] + timedelta(-1, i*slic*60)
    new_data = pd.merge(data, data_copy, left_on=['tag',s], right_on=['tag','time'], how='left')
    new_data[s] = new_data['count']
    data = new_data.drop(['time', 'count'], axis=1)
    del new_data
    gc.collect()

#时间片历史均值
# data = pd.merge(data, pd.DataFrame(wifi_2D['avg_count'].unstack()), left_on='timeStamp', right_index=True)
#当前ｔａｇ前５个时间片到后５个时间片的历史均值
wifi_m = wifi_mean[['timeStamp', 'tag', 'slice_mean']]
wifi_m.columns = ['time', 'tag', 'slice_mean']
for i in range(-8, 9):
    s='pre_'+str(i*slic)+'_min_avg'
    data[s] = data['timeStamp'] + timedelta(-1, i*slic*60)
    new_data = pd.merge(data, wifi_m, left_on=['tag',s], right_on=['tag','time'],how='left')
    new_data[s] = new_data['slice_mean']
    data = new_data.drop(['slice_mean','time'], axis=1)
    del new_data
    gc.collect()

#当前ｔａｇ前５到后５时间片标准差
wifi_m = wifi_mean[['timeStamp', 'tag', 'slice_std']]
wifi_m.columns = ['time', 'tag', 'slice_std']
for i in range(-8, 9):
    s='pre_'+str(i*slic)+'_min_std'
    data[s] = data['timeStamp'] + timedelta(-1, i*slic*60)
    new_data = pd.merge(data, wifi_m, left_on=['tag',s], right_on=['tag','time'],how='left')
    new_data[s] = new_data['slice_std']
    data = new_data.drop(['slice_std','time'], axis=1)
    del new_data
    gc.collect()

In [13]:
#聚类特征
# da = pd.read_excel('da.xls')
# clus = [7] + list(da.loc[4].values)
# clus = [int(c) for c in clus]
# cols = list(wifi_mean.groupby(['index', 'tag']).mean().unstack()['passengerCount'].columns)
# grou = []
# for i in range(len(group)):
#     grou.append(cols[clus[i]])
# data['grou'] = data['tag'].apply(lambda x:1 if x in grou else 0) 

# data = data.reset_index()
from sklearn.cluster import KMeans
km = KMeans(n_clusters=10)
wifi_group = km.fit_predict(wifi_corr)

dct = {}
for i in range(len(wifi_group)):
    dct[wifi_2D.columns[i]] = wifi_group[i]

# wifi_2D['avg_count'] = wifi_2D.mean(axis=1)

data['group'] = data['tag'].apply(lambda x:dct[x])
from sklearn.preprocessing import OneHotEncoder
oec = OneHotEncoder()

data['area'] = data['tag'].apply(lambda x:hash(x[:2])%1000)
data['mark'] = data['tag'].apply(lambda x:hash(x[4:5])%1000)
data['floor'] = data['tag'].apply(lambda x:hash(x[3:4])%1000)

oec.fit_transform(data[['area', 'mark', 'floor']]).toarray()
data = pd.concat([data, pd.DataFrame(oec.fit_transform(data[['group', 'area', 'mark', 'floor']]).toarray(), index=data.index)], axis=1)

data = data.drop(['group','area','mark', 'floor'],axis=1).fillna(0)

In [18]:
data = data.set_index('timeStamp')
data = data.fillna(0)

In [19]:
models = 3
predict_lst = []
for round in range(models):
    result = data.loc[date[-1][180/slic:], ['tag','passengerCount']]
    predict = result.copy()
    data_copy = data.copy()
    if offline>0:
        for d in date[-1][180/slic:]:
            data_copy.loc[d, 'passengerCount'] = 0
    import xgboost as xgb
    from sklearn.cross_validation import train_test_split
    feat = np.setdiff1d(data.columns, ['tag', 'timeStamp', 'passengerCount'])
    # feat = ['pre_10_min']
    for i in range(180/slic):
    #180/slic
        print i
        train_index = []
        for j in range(len(date)-2):
    #         train_index = [date[j+1][180/slic+i]]
            train_index.append(date[j+1][180/slic+i])
        test_index = [date[-1][180/slic+i]]
        X_train,X_valid,y_train,y_valid = train_test_split(data_copy.loc[train_index,feat],
                                                          data_copy.loc[train_index,'passengerCount'], test_size=0.15)
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dvalid = xgb.DMatrix(X_valid, label=y_valid)


        dtest = xgb.DMatrix(data_copy.loc[test_index,feat])
        param = {'bst:max_depth':5, 'bst:eta':0.05, 'silent':0, 'objective':'reg:linear'}
        evallist  = [(dtrain,'train'),(dvalid,'valid')]
        num_round = 100
        bst = xgb.train( param, dtrain, num_round, evals=evallist, early_stopping_rounds=10)

        pred = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)
        #修改下一个模型的变量
        data_copy.loc[test_index,'passengerCount'] = pred
        for j in range(1, 11 if i<7 else (180/slic-i)):
            data_copy.loc[test_index[0] + timedelta(0,slic*j*60),'pre_-'+str(j*slic)+'_min'] = np.array(pred).reshape(-1,1)
#             print data_copy.loc[test_index[0] + timedelta(0,slic*j*60),'pre_-'+str(j*slic)+'_min']
#             print data_copy.loc[test_index[0] + timedelta(0,slic*j*60)]
        predict.loc[test_index,'passengerCount'] = pred
    #     import os
    #     predict.loc[test_index].resample('10T').sum().plot()
    #     data.resample('10T').sum()['passengerCount'].plot()
    predict_lst.append(predict)

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


0
[0]	train-rmse:4.70963	valid-rmse:4.93309
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 10 rounds.
[1]	train-rmse:3.49683	valid-rmse:3.68266
[2]	train-rmse:2.68254	valid-rmse:2.8595
[3]	train-rmse:2.15587	valid-rmse:2.35475
[4]	train-rmse:1.81787	valid-rmse:2.05578
[5]	train-rmse:1.60869	valid-rmse:1.8915
[6]	train-rmse:1.47393	valid-rmse:1.81495
[7]	train-rmse:1.39294	valid-rmse:1.75645
[8]	train-rmse:1.34349	valid-rmse:1.74511
[9]	train-rmse:1.29429	valid-rmse:1.73656
[10]	train-rmse:1.25476	valid-rmse:1.7304
[11]	train-rmse:1.22225	valid-rmse:1.74237
[12]	train-rmse:1.18911	valid-rmse:1.73978
[13]	train-rmse:1.17047	valid-rmse:1.73955
[14]	train-rmse:1.14642	valid-rmse:1.73828
[15]	train-rmse:1.13161	valid-rmse:1.73706
[16]	train-rmse:1.11685	valid-rmse:1.73539
[17]	train-rmse:1.0994	valid-rmse:1.7294
[18]	train-rmse:1.08278	valid-rmse:1.73352
[19]	train-rmse:1.06341	valid-rmse:1.7371
[20]	trai

In [20]:
predict=predict_lst[-1].reset_index()
for i in range(models-1):
    predict = pd.merge(predict, predict_lst[i].reset_index(), on=['tag', 'timeStamp'])
    predict['passengerCount'] = (predict['passengerCount_x']+predict['passengerCount_y'])
    predict = predict.drop(['passengerCount_x', 'passengerCount_y'], axis=1)
predict['passengerCount']/=float(models)
predict = predict.set_index('timeStamp')

In [21]:
from sklearn.metrics import mean_squared_error
print mean_squared_error(result['passengerCount'], predict['passengerCount'])*(predict.shape[0]+8)

grouped = result.groupby('tag')
g_lst = []
for tag, group in grouped:
    group = group.resample('10T').mean()
    group['tag'] = tag
    g_lst.append(group)
result1 = pd.concat(g_lst)

grouped = predict.groupby('tag')
g_lst = []
for tag, group in grouped:
    group = group.resample('10T').mean()
    group['tag'] = tag
    g_lst.append(group)
predict1 = pd.concat(g_lst)

tmp = pd.merge(predict1.reset_index(), result1.reset_index(), on=['timeStamp', 'tag'])
from sklearn.metrics import mean_squared_error
mean_squared_error(tmp['passengerCount_x'], tmp['passengerCount_y'])*(tmp.shape[0]+8)

98137.7212744


98137.721274391428

In [ ]:
print np.sum(tmp['passengerCount_x'])
np.sum(tmp['passengerCount_y'])

In [ ]:
x = pd.read_csv("x.csv")
j = pd.read_csv("j.csv")

In [ ]:
result = predict.reset_index()
result['slice10min'] = result['timeStamp'].apply(lambda x:str(x)[:10]+'-'+str(x)[11:13]+'-'+str(x.minute/10))
result=result.drop('timeStamp', axis=1)

result.columns=['WIFIAPTag', 'passengerCount', 'slice10min']

mer = pd.merge(x, result, on=['WIFIAPTag', 'slice10min'], how='left')

mer.loc[mer['passengerCount_y'].isnull(), 'passengerCount_y'] = mer.loc[mer['passengerCount_y'].isnull(), 'passengerCount_x']
print mean_squared_error(mer['passengerCount_x'], mer['passengerCount_y'])*mer.shape[0]
print mer.corr()

result=mer.drop('passengerCount_x', axis=1)
result.columns=['WIFIAPTag', 'slice10min', 'passengerCount']

In [ ]:
result.to_csv("result.csv", index=False, columns=['passengerCount', 'WIFIAPTag', 'slice10min'])

In [ ]:
result.to_csv('model_18.csv', index=False)

In [ ]:
np.unique(predict['WIFIAPTag']).shape

In [ ]:
print np.sum(tmp['passengerCount_x'])
np.sum(tmp['passengerCount_y'])

In [ ]:
bst.get_fscore()

In [ ]:
data_copy.resample('10T').sum()['passengerCount'].plot()
data.resample('10T').sum()['passengerCount'].plot()

In [ ]:
for tag,group in wifi.groupby('WIFIAPTag'):
    group.set_index('timeStamp').resample('5T').sum().reindex(date).fillna(0).plot()
    savefig('fig/'+tag+'.jpg')
    plt.close()

In [ ]:
data_copy[data_copy['tag']=='E1-1A-1<E1-1-01>']['passengerCount'].plot()
data[data_copy['tag']=='E1-1A-1<E1-1-01>']['passengerCount'].plot()

In [ ]:
test = test.reset_index()
test['slice10min'] = test['timeStamp'].apply(lambda x:str(x)[:10]+'-'+str(x)[11:13]+'-'+str(x.minute/10))
test=test.drop('timeStamp', axis=1)
test.to_csv("result.csv", index=False)

In [ ]:
#offline预测１4日的12-14
#online预测１４日的１５－１７

train_index = list(date2) + list(date3[:30])
valid_index = list(date3[30:])
if offline:
    for d in train_index:

In [ ]:
gate = pd.read_csv("airport_gz_gates.csv")
flight = pd.read_csv("airport_gz_flights_chusai_1stround.csv", parse_dates=[1,2])
secu = pd.read_csv("airport_gz_security_check_chusai_1stround.csv", parse_dates=[1])
flight = pd.merge(flight, gate, how='left').sort_values(['flight_ID', 'scheduled_flt_time'])
print flight.shape

from datetime import timedelta
# flight = flight[flight['actual_flt_time'].notnull()]
flight['scheduled_flt_time'] = flight['scheduled_flt_time'].apply(lambda x:x+timedelta(0,8*60*60))
flight['actual_flt_time'] = flight['actual_flt_time'].apply(lambda x:x+timedelta(0,8*60*60))
#飞机提前起飞，可能是全部人员提前登机
flight.loc[flight['actual_flt_time']<flight['scheduled_flt_time'], 'actual_flt_time'] = \
    flight.loc[flight['actual_flt_time']<flight['scheduled_flt_time'], 'scheduled_flt_time']
# flight = flight[flight['BGATE_ID'].notnull()]
# flight = flight[flight['BGATE_AREA'].notnull()]

In [ ]:
#得到每个安检的人具体飞机时间
secu = pd.merge(flight, secu)
secu = secu[secu['security_time']<secu['actual_flt_time']]
secu = secu.sort_values('actual_flt_time').drop_duplicates(['flight_ID','passenger_ID'])

secu['t'] = secu['scheduled_flt_time']-secu['security_time']
secu.loc[secu['t']<timedelta(0), 't'] = secu.loc[secu['t']<timedelta(0), 'actual_flt_time']-secu.loc[secu['t']<timedelta(0), 'security_time']
#提前安检的时间分布
secu['t'].describe()

In [ ]:
#假设每个人都在起飞前２５分钟登机
secu['actual_flt_time'] = secu['actual_flt_time'].apply(lambda x:x-timedelta(0,25*60))

secu = secu.loc[secu['actual_flt_time']<'2016-09-14 15:00:00']

In [ ]:
#得到每个时间点走多少人
out_minute = secu.set_index('actual_flt_time').resample('T').count()

In [ ]:
#每个小时走了多少人
out_hour = out_minute.resample('10T').sum()['flight_ID'].plot()

In [ ]:
secu_W1 = secu[secu['BGATE_AREA']=='W1']
out_minute = secu_W1.set_index('actual_flt_time').resample('T').count()
out_10minute = out_minute.resample('10T').sum()['flight_ID']
out_10minute.plot()

In [ ]:
index = np.union1d(in_10minute.index, out_10minute.index)
(in_10minute.reindex(index).fillna(0) - out_10minute.reindex(index).fillna(0)).cumsum().plot()

In [ ]:
in_minute = secu_W1.set_index('security_time').resample('T').count()
in_10minute = in_minute.resample('10T').sum()['flight_ID']
in_10minute.plot()

In [ ]:
(in_10minute - out_10minute).cumsum()

In [ ]:
in_10minute.cumsum()

In [ ]:
wifi_W1 = wifi[wifi['area']=='W1']
wifi_W1.set_index('timeStamp').resample('10T').mean().plot()

In [ ]:
grouped = flight.groupby('BGATE_AREA')
g_lst = []
for area, group in grouped:
    print area
    g_lst.append(group.set_index('actual_flt_time').resample('1d').count())

In [ ]:
wifi_secu = wifi[(wifi['loca']=='E1') | (wifi['loca']=='E2') | (wifi['loca']=='E3') | (wifi['loca']=='W1') | (wifi['loca']=='W2') 
     | (wifi['loca']=='W3') | (wifi['loca']=='EC') | (wifi['loca']=='WC')]

In [ ]:
wifi_secu.set_index('timeStamp')['2016-09-11':].resample('10T').sum().plot()

In [ ]:
w1_flight = flight[flight['BGATE_AREA']=='W1'].set_index('actual_flt_time')
w1_flight_11 = w1_flight['2016-09-11':'2016-09-11'].reset_index()
w1_flight_12 = w1_flight['2016-09-12':'2016-09-12'].reset_index()

In [ ]:
secu['']

In [ ]:
max(w1_flight_11.index)

In [ ]:
g_lst[1].head()

In [ ]:
secu.set_index('security_time').resample('10t').count().plot()

secu[secu['flight_ID']=='CZ3381'].shape

In [ ]:
flight[flight['BGATE_AREA']=='W1'].sort_values('actual_flt_time')

In [ ]:
passCount['2016-09-10 23:00:00':]

In [ ]:
# t11 = '2016-09-11 15:00:00':'2016-09-11 17:50:00'
# t12 = '2016-09-12 15:00:00':'2016-09-12 17:50:00'
# t13 = '2016-09-13 15:00:00':'2016-09-13 17:50:00'
g_lst[1]['2016-09-11 15:00:00':'2016-09-11 17:50:00']

In [ ]:
T1.set_index('timeStamp').resample('60t').sum().plot()

In [ ]:
passCount.plot()

In [ ]:
#E1 47
#E2 48
#E3 41
#W1 53
#W2 42
#W3 40
#登机口

#EC 152
#WC  158
#安检中

#T1 165
#安检前